In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.image as plm
import seaborn as sns
import folium
from folium import IFrame
import shutil
#convert html to png
import selenium.webdriver
import time

os.chdir("D:/gps/")
# merge_data 作为主要的数据集
m_data = pd.read_pickle("merged_clean_gps.pkl",)
# 将所有日期转化为同一天
m_data.index = pd.to_datetime("2012-11-12 "+ m_data.Time.dt.time.astype("str").str[:])
la = m_data.Latitude.median()
lo = m_data.Longitude.median()
m_data.head()

,Time,Latitude,Longitude,ID
Time,,,,
2012-11-12 07:47:57,2012-11-12 07:47:57,35.019531,135.780149,ETSS00001
2012-11-12 07:59:25,2012-11-12 07:59:25,35.018592,135.780094,ETSS00001
2012-11-12 08:01:03,2012-11-12 08:01:03,35.018605,135.778936,ETSS00001
2012-11-12 08:03:02,2012-11-12 08:03:02,35.017700,135.778466,ETSS00001
2012-11-12 08:05:03,2012-11-12 08:05:03,35.017524,135.777437,ETSS00001


In [2]:
#main function, used for spliting target area
def split_area(la_min=34.874676
               ,la_max=35.077540
               ,lo_min=135.663744
               ,lo_max=135.837235
               ,h_n=15,w_n=10):
    
    #get h_n+1 point in la region
    each_la = np.linspace(la_max,la_min,h_n+1).reshape(-1,1)
    #repeat w_n+1 times in axis=1 
    all_la = np.tile(each_la,(1,w_n+1))    
    each_lo = np.linspace(lo_min,lo_max,w_n+1).reshape(1,-1)
    all_lo = np.tile(each_lo,(h_n+1,1))
    all_loci = np.dstack((all_la,all_lo))
    
    rsp = lambda x : x[:2]+x[2:][::-1] #reshape point
    each_cell = []
    for i in range(h_n):
        for j in range(w_n):
            sub_cell = all_loci[i:i+2,j:j+2].copy().reshape(4,2).tolist()
            each_cell.append(rsp(sub_cell))
    
    each_cell = np.array(each_cell).reshape(h_n,w_n,4,2)
    region_dict = {(i,j):each_cell[i,j].tolist() for i in range(h_n) for j in range(w_n)}  # transfer to dict
    
    return region_dict,((each_la.flatten()[::-1],list(range(h_n))[::-1]),#!!! 经度的命名为反向排列
                      (each_lo.flatten(),list(range(w_n))))

# split_dict,labels = split_area()

In [3]:
#determine point location in splited area
def annotation_for_m(m_data,labels):
    m_data["La_cut"] = pd.cut(m_data.Latitude,bins =labels[0][0],labels=labels[0][1])
    m_data["Lo_cut"] = pd.cut(m_data.Longitude,bins = labels[1][0],labels=labels[1][1])
    m_data["region_label"] = m_data.apply(lambda x: (x["La_cut"],x["Lo_cut"]),axis=1)
    return m_data

# m_data = annotation_for_m(m_data,labels)

In [23]:
#counts tourists number(without duplicates) in different area and different period
def region_resample(input_array,freq = "H"):
    data_list = []
    region_list = input_array.region_label.unique()
    total_i = len(region_list)
    for i,each in enumerate(region_list):
        sub_region = input_array.loc[input_array.region_label==each]
        temp = sub_region.resample(freq)[["ID"]].nunique()
        temp.loc[temp.ID>=150,["ID"]]=149 #超过150人的区域  标记为150
#         return temp
        temp["region_label"] = str(each)
        data_list.append(temp)
        print(f"\r{i+1}/{total_i} regions have been processed!",end="")
        
    return pd.concat(data_list,axis=0).dropna()


# time_region_res = region_resample(m_data,freq = "H")

In [10]:
# convert gps data to json format for plot
def dict_2_geojson(in_dict):
    #change the sequence of Lo and La, add the first point
    out_dict = {k:[i[::-1] for i in v ] + [v[0][::-1]] for k,v in in_dict.items()}
    def return_json(id,coordinate):
        return {
            "type": "Feature",
            "properties": {
              "DISTRICT": id,
            },
            "geometry": {
              "type": "Polygon",
              "coordinates": [coordinate

              ]
            }
        }
    features_list = [return_json(str(k),v) for k,v in out_dict.items()]
    return {  "type": "FeatureCollection",  "features": features_list}

# geo_data = dict_2_geojson(split_dict)

In [11]:
# plot heatmap in folium map
def coloropleth(geo_data,data):
    m = folium.Map(location=[la-0.02,lo], zoom_start=11,
#         width="50%",
                   )
    folium.Choropleth(
        geo_data=geo_data,
        data=data,
        columns=["region_label","ID"],
        key_on='feature.properties.DISTRICT',
        #fill_color='red',
        fill_color='YlOrRd',
        fill_opacity=0.5,
        line_opacity=0.5,
        legend_name = "Counts of tourists",
        nan_fill_color ="grey",
        bins = list(range(0,151,30)),
    ).add_to(m)
    return m

# res_map = coloropleth(geo_data,time_region_res.loc[time_region_res.index == "2012-11-12 12:00:00"])
# res_map

In [33]:
#verify the function of split_area
s,labels = split_area(34.874676,35.077540,135.663744,135.837235,15,10)
m = folium.Map(location=[la,lo], zoom_start=11)
for each in s.values():
    folium.Polygon(each,color='blue', weight=2, fill=True,fill_color='blue', fill_opacity=0).add_to(m)
folium.Polygon(s[(2,5)],color='blue', weight=2, fill=True,fill_color='red', fill_opacity=1).add_to(m)
m

In [35]:
# region label，sequence is similar to numpy
pd.DataFrame(np.array([str([i,j]) for i in range(15) for j in range(10)]).reshape(15,10))

,0,1,2,3,4,5,6,7,8,9
0,"[0, 0]","[0, 1]","[0, 2]","[0, 3]","[0, 4]","[0, 5]","[0, 6]","[0, 7]","[0, 8]","[0, 9]"
1,"[1, 0]","[1, 1]","[1, 2]","[1, 3]","[1, 4]","[1, 5]","[1, 6]","[1, 7]","[1, 8]","[1, 9]"
2,"[2, 0]","[2, 1]","[2, 2]","[2, 3]","[2, 4]","[2, 5]","[2, 6]","[2, 7]","[2, 8]","[2, 9]"
3,"[3, 0]","[3, 1]","[3, 2]","[3, 3]","[3, 4]","[3, 5]","[3, 6]","[3, 7]","[3, 8]","[3, 9]"
4,"[4, 0]","[4, 1]","[4, 2]","[4, 3]","[4, 4]","[4, 5]","[4, 6]","[4, 7]","[4, 8]","[4, 9]"
5,"[5, 0]","[5, 1]","[5, 2]","[5, 3]","[5, 4]","[5, 5]","[5, 6]","[5, 7]","[5, 8]","[5, 9]"
6,"[6, 0]","[6, 1]","[6, 2]","[6, 3]","[6, 4]","[6, 5]","[6, 6]","[6, 7]","[6, 8]","[6, 9]"
7,"[7, 0]","[7, 1]","[7, 2]","[7, 3]","[7, 4]","[7, 5]","[7, 6]","[7, 7]","[7, 8]","[7, 9]"
8,"[8, 0]","[8, 1]","[8, 2]","[8, 3]","[8, 4]","[8, 5]","[8, 6]","[8, 7]","[8, 8]","[8, 9]"
9,"[9, 0]","[9, 1]","[9, 2]","[9, 3]","[9, 4]","[9, 5]","[9, 6]","[9, 7]","[9, 8]","[9, 9]"


In [24]:
# main function
path = "./heatmap_2_time_series/"
if not os.path.exists(path):
    os.mkdir(path)
    
driver = selenium.webdriver.PhantomJS()
driver.set_window_size(720,480) 

split_dict,labels = split_area(h_n=20,w_n=13)

m_data = annotation_for_m(m_data,labels)

geo_data = dict_2_geojson(split_dict)

for period in ["10min","20min","30min","1H","2H","3H"]:
    time_region_res = region_resample(m_data,freq = period)

    time_list = sorted((time_region_res.index.unique()))
    time_n = len(time_list)
    for i,each in enumerate(time_list):    
        res_map = coloropleth(geo_data,time_region_res.loc[time_region_res.index == str(each)])
        
        name = path+f"{period}_{str(each).replace(':','-')}"
        res_map.save(f'{name}.html')

        driver.get(f'{name}.html')
        # sleep(seconds) here
        time.sleep(1)
        driver.save_screenshot(f'{name}.png')
        print(f"\r{i+1}/{time_n} in {period} have been processed!",end="")

C:\conda\envs\python3\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


8/8 in 3H have been processed!essed!

In [29]:
#由于未知原因导致截图的图片的图例为纯黑色，通过plt 自动添加
def add_cmap():
    c_map = plm.imread("D:/gps/heatma_cmap.PNG")
    c_map = np.array(c_map)
    c_map[c_map==0.8666667] = 1.0
    
    path = "D:heatmap_2_time_series/"
    file_list = [i for i in os.listdir(path) if i.endswith("png")]
    for file in file_list:
        map_fig = plt.imread(path+file)
        map_fig = np.array(map_fig)[50:,100:-130]
        fig,ax = plt.subplots(dpi=300,)
        ax.imshow(map_fig)
        ax.axis("off")
        bar = fig.add_axes([0.25,0.89,0.5,0.1])
        bar.imshow(c_map)
        bar.axis("off")
        plt.savefig(path+"with_cbar_"+file)
        plt.close()
add_cmap()

In [32]:
def build_gif(path,prefix,out="gif"):
    """function to create a gif of heatmaps"""
    file_list = [i for i in os.listdir(path) if i.startswith("with_cbar_"+prefix) and i.endswith(".png")]
    rgbs = [plm.imread(path+i) for i in file_list]
    time_list = [i.split(" ")[1].replace("-",":").rstrip(".png") for i in file_list]
    fig, ax = plt.subplots(dpi=150 if out=="gif" else 300)
    ax.imshow(rgbs[0])
    ax.set_axis_off()
    def show_im(pairs):
        ax.clear()
        ax.set_title(pairs[0])
        ax.imshow(pairs[1])
        ax.set_axis_off() 
    pairs = list(zip(time_list,rgbs))
    #ims = map(lambda x: (ax.imshow(x), ax.set_title(title)), imgs)
    im_ani = animation.FuncAnimation(fig, show_im, pairs,interval=1000 if "H" in prefix else 500
                                     , repeat_delay=0, blit=False)
    plt.cla()
    im_ani.save(path+f'{prefix}_heatmap_animation.{out}',writer= "pillow" if out=="gif" else 'ffmpeg') #, writer='imagemagick'
    plt.close()

for p in ["10min","20min","30min","1H","2H","3H"]:
    build_gif("./heatmap_2_time_series/",p,"mp4")
    build_gif("./heatmap_2_time_series/",p,"gif")

In [40]:
# import shutil
# for p in ["10min","20min","30min","1H","2H","3H"]:
#     path = "D:/gps/heatmap_2_time_series/"
#     target_path = f"{path}every_{p}/"
#     if not os.path.exists(target_path):
#         os.mkdir(target_path)
#     for file in [i for i in os.listdir(path) if p in i]:
#         shutil.move(path+file,target_path)
# #         print(path+file,target_path)